In [ ]:
with open('review-Rhode_Island.json', 'r') as file:
    lines = [next(file) for _ in range(5)]  # Read the first 5 lines

# Print the lines to inspect them
for line in lines:
    print(line)

{"user_id": "102238134963500607023", "name": "Majestically child", "time": 1600000495003, "rating": 5, "text": "Always wanted to check the place out to see of their food was any good and I finally had the opportunity yesterday. It didn't disappoint ! It was sooo good ! I only had their lasagna but I already can't wait to go back and try everything else. Very friendly service and very clean. Also love the set up inside and outside \ud83d\ude0a", "pics": null, "resp": {"time": 1602371908205, "text": "Thanks for coming hope to see you soon"}, "gmap_id": "0x89e44fe1c2a2a2c3:0x5fc9f6751fb29737"}

{"user_id": "108990070407716626305", "name": "Kathryn Bartizek", "time": 1602191331373, "rating": 5, "text": "Mofongo, spanish sausage, chicken stew, rice, all were fabulous!", "pics": [{"url": ["https://lh5.googleusercontent.com/p/AF1QipNn1cf3m9UZX1jYzPgf47pUwHV6b7gSCdOM2_-e=w150-h150-k-no-p"]}], "resp": {"time": 1602371794024, "text": "Very glad to heard that hope to see you back soon. Thanks for

In [ ]:
import json
from collections import defaultdict, Counter
import numpy as np

# Function to load and clean the JSON data
def load_and_clean_json(file_path):
    cleaned_data = []
    total_rows = 0
    with open(file_path, 'r') as file:
        for line in file:
            total_rows += 1
            data = json.loads(line.strip())
            if data['user_id'] is not None and data['text'] is not None and data['gmap_id'] is not None:
                cleaned_data.append(data)
    return total_rows, cleaned_data

# Function to count reviews per gmap_id and calculate distribution of these counts
def count_reviews_per_gmap_id(cleaned_data):
    review_counts = defaultdict(int)
    for data in cleaned_data:
        review_counts[data['gmap_id']] += 1

    # Calculate how many gmap_ids have the same number of reviews
    distribution = Counter(review_counts.values())
    total_gmap_ids = sum(review_counts.values())  # Total gmap_ids reviewed

    # Calculate the percentage of total for each unique count of reviews
    distribution_percentages = {count: (num_gmap_ids / total_gmap_ids) * 100 for count, num_gmap_ids in distribution.items()}
    return review_counts, distribution, distribution_percentages

# Specify the path to your JSON file
file_path = '/content/review-Rhode_Island.json'

# Load and clean the JSON data
total_rows, cleaned_data = load_and_clean_json(file_path)
review_counts, distribution_of_reviews, distribution_percentages = count_reviews_per_gmap_id(cleaned_data)

# Print the results
print("Total rows in the original dataset:", total_rows)
print("Number of rows after cleaning:", len(cleaned_data))
print("\nReview count per gmap_id:")
for gmap_id, count in review_counts.items():
    print(f"gmap_id {gmap_id}: {count} reviews")

print("\nDistribution of review counts across gmap_ids with percentages:")
for review_count, gmap_count in sorted(distribution_of_reviews.items()):
    percentage = distribution_percentages[review_count]
    print(f"{gmap_count} gmap_ids with {review_count} reviews ({percentage:.5f}%)")


Streaming output truncated to the last 5000 lines.
gmap_id 0x89e44b0ca876d62f:0xae1a87379dfeaff5: 20 reviews
gmap_id 0x89e44b0470c20033:0xfcccf87f6f834d20: 105 reviews
gmap_id 0x89e44b4ca9e9dfbb:0x84915c693d602298: 82 reviews
gmap_id 0x89e44d2963a3ce57:0xae8eb17e47eeb2aa: 70 reviews
gmap_id 0x89e45bdba7539f8f:0xebf06648d18532d1: 57 reviews
gmap_id 0x89e44512a14ad291:0xcde3b43454f3ac3: 35 reviews
gmap_id 0x89e5af5c4fb80ab7:0xca5bcac7f0375fc6: 23 reviews
gmap_id 0x89e5af66e69e4eb3:0x89dc4cfda3602f29: 17 reviews
gmap_id 0x89e5e767f63b87cd:0x136c93a912149497: 109 reviews
gmap_id 0x89e5e715d8ef02bb:0x93a8baf93b00214f: 46 reviews
gmap_id 0x89e44c4ff328896f:0x4bdd963801cbedcc: 226 reviews
gmap_id 0x89e5e7029319eea3:0x9115cf5b6f262b76: 45 reviews
gmap_id 0x89e5b7ed09ec2df1:0x2f2aa732120a3dbd: 16 reviews
gmap_id 0x89e5e7752e518a9f:0xcd38868c1726b08: 46 reviews
gmap_id 0x89e5b373c34d986f:0x470704575f644b36: 35 reviews
gmap_id 0x89e5e7170423bf1d:0x3d55efce4959d03a: 34 reviews
gmap_id 0x89e5e71f26

In [ ]:
num_users_train = 20000
num_users_dev = 2500
num_users_test = 2500



from collections import defaultdict


# Transforming cleaned_data into a dictionary of user_ids to list of reviews
user_reviews_dict = defaultdict(list)
for review in cleaned_data:
    user_reviews_dict[review['user_id']].append(review)

# Convert dictionary to list of users (as expected by the existing code)
users_list = list(user_reviews_dict.items())

# Now you can proceed with the existing function to build global product counts
def build_global_product_counts(users):
    product_count = defaultdict(list)  # Store list of users who reviewed each product
    for user_id, reviews in users:
        # Focus on the first review in each user's profile
        first_review = reviews[0]
        product_count[first_review['gmap_id']].append(user_id)  # Track which users reviewed the first product
    return product_count

# Global product counts for the entire dataset
global_product_counts = build_global_product_counts(users_list)




In [ ]:
from collections import defaultdict

# Initialize a dictionary to keep track of how many users reviewed each product based on their first review's gmap_id
global_neighbor_counts = defaultdict(int)

for user_id, user_reviews in user_reviews_dict.items():
    if user_reviews:  # Ensure there is at least one review
        first_review_gmap_id = user_reviews[0]['gmap_id']  # Focus on the gmap_id of the first review
        global_neighbor_counts[first_review_gmap_id] += 1  # Increment count for this gmap_id globally

def print_global_neighbor_distribution_with_percentages(global_neighbor_counts, split_name):
    # Calculate the distribution of neighbor counts
    neighbor_count_distribution = defaultdict(int)

    for gmap_id, count in global_neighbor_counts.items():
        neighbor_count_distribution[count] += 1

    # Calculate total number of gmap_ids
    total_gmap_ids = sum(neighbor_count_distribution.values())

    # Print the distribution with percentages
    print(f"Global Neighbor Count Distribution for {split_name} split (based on first review):")
    sorted_neighbor_distribution = sorted(neighbor_count_distribution.items(), key=lambda x: x[0])

    for neighbor_count, gmap_id_count in sorted_neighbor_distribution:
        percentage = (gmap_id_count / total_gmap_ids) * 100
        print(f"Gmap_ids with {neighbor_count} reviews: {gmap_id_count} ({percentage:.2f}%)")
    print("\n")

# Example usage of the function, assuming the split name you want to use
print_global_neighbor_distribution_with_percentages(global_neighbor_counts, "Global Neighbors (First Review)")


Global Neighbor Count Distribution for Global Neighbors (First Review) split (based on first review):
Gmap_ids with 1 reviews: 1278 (8.91%)
Gmap_ids with 2 reviews: 1126 (7.85%)
Gmap_ids with 3 reviews: 975 (6.80%)
Gmap_ids with 4 reviews: 844 (5.89%)
Gmap_ids with 5 reviews: 700 (4.88%)
Gmap_ids with 6 reviews: 681 (4.75%)
Gmap_ids with 7 reviews: 562 (3.92%)
Gmap_ids with 8 reviews: 505 (3.52%)
Gmap_ids with 9 reviews: 395 (2.75%)
Gmap_ids with 10 reviews: 417 (2.91%)
Gmap_ids with 11 reviews: 342 (2.38%)
Gmap_ids with 12 reviews: 372 (2.59%)
Gmap_ids with 13 reviews: 266 (1.85%)
Gmap_ids with 14 reviews: 276 (1.92%)
Gmap_ids with 15 reviews: 286 (1.99%)
Gmap_ids with 16 reviews: 260 (1.81%)
Gmap_ids with 17 reviews: 289 (2.02%)
Gmap_ids with 18 reviews: 228 (1.59%)
Gmap_ids with 19 reviews: 190 (1.32%)
Gmap_ids with 20 reviews: 166 (1.16%)
Gmap_ids with 21 reviews: 150 (1.05%)
Gmap_ids with 22 reviews: 132 (0.92%)
Gmap_ids with 23 reviews: 145 (1.01%)
Gmap_ids with 24 reviews: 162 (

In [ ]:
def set_random_first_review_with_global_neighbor(user_reviews, global_product_counts):
    # Filter reviews that have at least one globxal neighbor
    valid_reviews = [review for review in user_reviews if len(global_product_counts[review['gmap_id']]) > 1]

    if valid_reviews:
        random_review = random.choice(valid_reviews)  # Pick a random review with global neighbors
        user_reviews.remove(random_review)
        user_reviews.insert(0, random_review)
    else:
        return None  # No reviews with global neighbors, exclude the user

    return user_reviews

In [ ]:
def add_user_to_split(user_id, user_reviews, split_users, local_product_counts, added_users):
    # Add the user to the split
    split_users.append((user_id, user_reviews))
    added_users.add(user_id)
    # Update local product counts for the split
    for review in user_reviews:
        local_product_counts[review['gmap_id']] += 1

In [ ]:
import pandas as pd
import random

from collections import defaultdict
import tqdm
from datetime import datetime, timedelta
import logging
logging.basicConfig(level=logging.INFO)

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
def distribute_global_neighbors(users, num_users_train, num_users_dev, num_users_test, global_product_counts):
    train_users = []
    dev_users = []
    test_users = []

    # Initialize local product counts for train, dev, and test splits
    local_product_counts_train = defaultdict(int)
    local_product_counts_dev = defaultdict(int)
    local_product_counts_test = defaultdict(int)

    # Track which users have already been added
    added_users = set()

    # Exclude products with only one review
    filtered_product_list = [gmap_id for gmap_id, user_ids in global_product_counts.items() if len(user_ids) > 1]

    # Set the seed before shuffling to ensure reproducibility
    random.seed(RANDOM_SEED)
    random.shuffle(filtered_product_list)

    # Go through each product's global neighbors in a random order
    for gmap_id in filtered_product_list:
        user_ids = global_product_counts[gmap_id]

        # Set the seed before shuffling users to ensure reproducibility
        random.seed(RANDOM_SEED)
        random.shuffle(user_ids)

        # Add the first two users to the training set
        if len(train_users) < num_users_train - 1:
            for user_id in user_ids[:2]:  # Add the first two users to the training set
                if user_id in added_users:
                    continue  # Skip users who have already been added

                # Get the user's reviews
                user_reviews = [user for user in users if user[0] == user_id][0][1]

                # Add the user to the training split
                add_user_to_split(user_id, user_reviews, train_users, local_product_counts_train, added_users)

        # Add the next two users to the dev set if training set is filled
        elif len(dev_users) < num_users_dev - 1:
            for user_id in user_ids[:2]:  # Add the next two users to the dev set
                if user_id in added_users:
                    continue  # Skip users who have already been added

                # Get the user's reviews
                user_reviews = [user for user in users if user[0] == user_id][0][1]

                # Add the user to the dev split
                add_user_to_split(user_id, user_reviews, dev_users, local_product_counts_dev, added_users)

        # Add the next two users to the test set if dev set is filled
        elif len(test_users) < num_users_test - 1:
            for user_id in user_ids[:2]:  # Add the next two users to the test set
                if user_id in added_users:
                    continue  # Skip users who have already been added

                # Get the user's reviews
                user_reviews = [user for user in users if user[0] == user_id][0][1]

                # Add the user to the test split
                add_user_to_split(user_id, user_reviews, test_users, local_product_counts_test, added_users)

        # Add subsequent users for this product, ensuring they have local neighbors
        for i, user_id in enumerate(user_ids[2:], start=2):
            if user_id in added_users:
                continue  # Skip users who have already been added

            # Get the user's reviews
            user_reviews = [user for user in users if user[0] == user_id][0][1]

            # Ensure subsequent users have local neighbors in the train split
            if len(train_users) < num_users_train:
                if local_product_counts_train[gmap_id] > 0:  # Ensure local neighbor exists
                    add_user_to_split(user_id, user_reviews, train_users, local_product_counts_train, added_users)

            # Ensure subsequent users have local neighbors in the dev split
            elif len(dev_users) < num_users_dev:
                if local_product_counts_dev[gmap_id] > 0:  # Ensure local neighbor exists
                    add_user_to_split(user_id, user_reviews, dev_users, local_product_counts_dev, added_users)

            # Ensure subsequent users have local neighbors in the test split
            elif len(test_users) < num_users_test:
                if local_product_counts_test[gmap_id] > 0:  # Ensure local neighbor exists
                    add_user_to_split(user_id, user_reviews, test_users, local_product_counts_test, added_users)

            # Stop if all splits are full
            if len(train_users) >= num_users_train and len(dev_users) >= num_users_dev and len(test_users) >= num_users_test:
                break

    return train_users, dev_users, test_users, local_product_counts_train, local_product_counts_dev, local_product_counts_test
train_users, dev_users, test_users, local_product_counts_train, local_product_counts_dev, local_product_counts_test = distribute_global_neighbors(
    users_list, num_users_train, num_users_dev, num_users_test, global_product_counts
)


In [ ]:
logging.info(f"Number of users in the test set: {len(test_users)}")
logging.info(f"Number of users in the dev set: {len(dev_users)}")
# Function to get the profile size distribution
def get_profile_size_distribution(users):
    profile_sizes = [len(reviews) for user_id, reviews in users]
    return pd.Series(profile_sizes).value_counts().sort_index()

In [ ]:
def get_local_neighbor_count(users, local_product_counts):
    user_neighbor_counts = []

    for user_id, reviews in users:
        first_review = reviews[0]  # Focus on the first review in the profile
        gmap_id = first_review['gmap_id']
        local_neighbors = local_product_counts[gmap_id] - 1  # Subtract 1 to exclude the current review
        user_neighbor_counts.append((user_id, local_neighbors))

    return pd.Series([count for _, count in user_neighbor_counts]).value_counts().sort_index()


In [ ]:
def print_local_neighbor_distribution_with_percentages(local_neighbor_count, split_name):
    total_users = local_neighbor_count.sum()  # Total number of users

    # Print the distribution with percentages
    print(f"{split_name} Local Neighbor Count Distribution (for the first review):")
    for neighbor_count, count in local_neighbor_count.items():
        percentage = (count / total_users) * 100
        print(f"Users with {neighbor_count} neighbors: {count} ({percentage:.2f}%)")
    print("\n")

# Function to print the profile size distribution with percentages
def print_profile_size_distribution_with_percentages(profile_size_distribution, split_name):
    total_users = profile_size_distribution.sum()  # Total number of users

    # Print the distribution with percentages
    print(f"{split_name} Profile Size Distribution:")
    for profile_size, count in profile_size_distribution.items():
        percentage = (count / total_users) * 100
        print(f"Users with profile size {profile_size}: {count} ({percentage:.2f}%)")
    print("\n")


In [ ]:
train_local_neighbor_count = get_local_neighbor_count(train_users, local_product_counts_train)
dev_local_neighbor_count = get_local_neighbor_count(dev_users, local_product_counts_dev)
test_local_neighbor_count = get_local_neighbor_count(test_users, local_product_counts_test)

# Calculate the profile size distributions for the train, dev, and test sets
train_profile_size_distribution = get_profile_size_distribution(train_users)
dev_profile_size_distribution = get_profile_size_distribution(dev_users)
test_profile_size_distribution = get_profile_size_distribution(test_users)

# Print local neighbor count distributions with percentages for each split
print_local_neighbor_distribution_with_percentages(train_local_neighbor_count, "Train")
print_local_neighbor_distribution_with_percentages(dev_local_neighbor_count, "Dev")
print_local_neighbor_distribution_with_percentages(test_local_neighbor_count, "Test")

# Print profile size distributions with percentages for each split
print_profile_size_distribution_with_percentages(train_profile_size_distribution, "Train")
print_profile_size_distribution_with_percentages(dev_profile_size_distribution, "Dev")
print_profile_size_distribution_with_percentages(test_profile_size_distribution, "Test")

Train Local Neighbor Count Distribution (for the first review):
Users with 1 neighbors: 94 (0.47%)
Users with 2 neighbors: 125 (0.62%)
Users with 3 neighbors: 137 (0.69%)
Users with 4 neighbors: 152 (0.76%)
Users with 5 neighbors: 203 (1.01%)
Users with 6 neighbors: 211 (1.05%)
Users with 7 neighbors: 117 (0.58%)
Users with 8 neighbors: 126 (0.63%)
Users with 9 neighbors: 158 (0.79%)
Users with 10 neighbors: 198 (0.99%)
Users with 11 neighbors: 188 (0.94%)
Users with 12 neighbors: 193 (0.97%)
Users with 13 neighbors: 214 (1.07%)
Users with 14 neighbors: 202 (1.01%)
Users with 15 neighbors: 185 (0.92%)
Users with 16 neighbors: 183 (0.92%)
Users with 17 neighbors: 358 (1.79%)
Users with 18 neighbors: 198 (0.99%)
Users with 19 neighbors: 194 (0.97%)
Users with 20 neighbors: 147 (0.73%)
Users with 21 neighbors: 287 (1.44%)
Users with 22 neighbors: 211 (1.05%)
Users with 23 neighbors: 217 (1.08%)
Users with 24 neighbors: 20 (0.10%)
Users with 25 neighbors: 163 (0.81%)
Users with 26 neighbor

In [ ]:
import json
def process_user_data(users):
    out = []
    predictions = []
    for user_id, reviews in tqdm.tqdm(users):
        # Generate user input and output (as done before)
        user_entry = {"id": user_id}
        pred_entry = {"id": user_id, "output": reviews[0]['text']}

        # Add full user profile to user_entry
        user_entry["profile"] = [
            {
                'rating': d['rating'],

                'text': d['text'],
                # Uncomment the lines below if you want to include these fields
                # 'timestamp': d['timestamp'],
                # 'helpful_vote': d['helpful_vote'],
                # 'verified_purchase': d['verified_purchase'],
                 "gmap_id": d['gmap_id']
            }
            for d in reviews
        ]

        # Append entries to output lists
        out.append(user_entry)
        predictions.append(pred_entry)
    return out, predictions

# Logging info
logging.info("Processing train, test, and dev data")

# Process the train, test, and dev sets
out_train, predictions_train = process_user_data(train_users)
out_test, predictions_test = process_user_data(test_users)
out_dev, predictions_dev = process_user_data(dev_users)

# File name base (you can modify this as needed)
file_name_base = "google_local_reviews"

# Function to write data to files
def write_to_file(data, file_suffix):
    file_name = f"{file_name_base}_{file_suffix}.json"
    with open(file_name, "w") as file:
        json.dump(data, file, indent=4)

# Logging and writing the files
logging.info("Writing data to files")
write_to_file(out_train, "questions_train")
write_to_file(predictions_train, "outputs_train")
write_to_file(out_test, "questions_test")
write_to_file(predictions_test, "outputs_test")
write_to_file(out_dev, "questions_dev")
write_to_file(predictions_dev, "outputs_dev")

# Log completion
logging.info("Data successfully written to files.")


100%|██████████| 2500/2500 [00:00<00:00, 137952.37it/s]
